In [1]:
import os
import numpy as np
import cv2
import shutil

In [2]:
def mkdir(path):
 
	folder = os.path.exists(path)
 
	if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
		os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
		print("---  new folder...  ---")
		print("---  OK  ---")
 
	else:
		print("---  There is this folder!  ---")	

## 存储原始数据集

In [3]:
inputDataset = './inputDataset/'
file0 = "./tempDataset/step0-重新编号后的数据集/"
mkdir(file0) 

i = 0
for root, dirs, files in os.walk(inputDataset, topdown=True):
    for name in files:
        print((""+os.path.join(root, name)))
        img=cv2.imread((""+os.path.join(root, name)))
        i = i + 1
        cv2.imwrite(file0 + str(i) + '.bmp', img)
#     for name in dirs:
#         print(os.path.join(root, name))

---  new folder...  ---
---  OK  ---
./inputDataset/ITF14/picture_3.bmp
./inputDataset/ITF14/picture_9.bmp
./inputDataset/ITF14/picture_8.bmp
./inputDataset/ITF14/picture_1.bmp
./inputDataset/ITF14/picture_0.bmp
./inputDataset/ITF14/picture_6.bmp
./inputDataset/ITF14/picture_2.bmp
./inputDataset/ITF14/picture_4.bmp
./inputDataset/ITF14/picture_10.bmp
./inputDataset/ITF14/picture_11.bmp
./inputDataset/ITF14/picture_5.bmp
./inputDataset/ITF14/picture_7.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_13.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_20.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_5.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_6.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_18.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_11.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_15.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_7.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_17.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_19.bmp
./inputDataset/MSI无校验（开启一位校验码的时候仍可识读）/SD02_2

./inputDataset/EAN13-13mil/28cm/EAN13 13mil 28cm (23).bmp
./inputDataset/EAN13-13mil/28cm/EAN13 13mil 28cm (29).bmp
./inputDataset/EAN13-13mil/28cm/EAN13 13mil 28cm (3).bmp
./inputDataset/EAN13-13mil/28cm/EAN13 13mil 28cm (28).bmp
./inputDataset/EAN13-13mil/28cm/EAN13 13mil 28cm (25).bmp
./inputDataset/EAN13-13mil/28cm/EAN13 13mil 28cm (7).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (26).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (15).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (24).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (8).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (7).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (4).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (19).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (13).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (21).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (3).bmp
./inputDataset/EAN13-13mil/10cm/EAN13 13mil 10cm (5).bmp
./inputDataset/EAN13-

./inputDataset/EAN13-13mil/37cm-不解码/EAN13 13mil 37cm (33).bmp
./inputDataset/EAN13-13mil/37cm-不解码/EAN13 13mil 37cm (11).bmp
./inputDataset/EAN13-13mil/37cm-不解码/EAN13 13mil 37cm (45).bmp
./inputDataset/EAN13-13mil/37cm-不解码/EAN13 13mil 37cm (16).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (44).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (26).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (7).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (4).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (9).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (6).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (27).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (30).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (2).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (21).bmp
./inputDataset/EAN13-13mil/36cm-扫码40次-识读1次/EAN13 13mil 36cm (28).bm

./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (47).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (42).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (6).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (12).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (26).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (2).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (50).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (23).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (44).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (41).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (32).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (29).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (35).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (43).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (16).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (15).bmp
./inputDataset/EAN13-13mil/16cm/EAN13 13mil 16cm (8).bmp
./inputDataset/EA

./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (48).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (13).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (20).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (18).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (16).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (10).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (2).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (12).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (54).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (47).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (43).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (26).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (11).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (58).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (57).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (30).bmp
./inputDataset/EAN13-13mil/25cm/EAN13 13mil 25cm (22).bmp
./inputDataset/

...

./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (83).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (109).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (85).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (19).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (32).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (1).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (77).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (95).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (104).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (66).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (92).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (43).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (76).bmp
./inputDataset/EAN13-13mil/33cm-扫码40次-识读28次/EAN13 13mil 33cm (80).bmp
./inputDataset/EAN1

./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (38).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (79).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (71).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (36).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (68).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (67).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (72).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (62).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (47).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (45).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (65).bmp
./inputDataset/EAN13-13mil/35cm-扫码40次-识读4次/EAN13 13mil 35cm (81).bmp
./inputDataset/EAN13-13mil/22cm/EAN13 13mil 22cm (46).bmp
./inputDataset/EAN13-13mil/22cm/EAN13 13mil 22cm (44).bmp
./inputDataset/EAN13-13mil/22cm/EAN13 13mil 22cm (26).bm

./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (23).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (20).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (32).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (11).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (21).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (40).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (15).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (29).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (8).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (14).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (9).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (48).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (37).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (18).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (25).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (49).bmp
./inputDataset/EAN13-13mil/23cm/EAN13 13mil 23cm (46).bmp
./inputDataset/E

./inputDataset/EAN13-13mil/14cm/EAN13 13mil 14cm (5).bmp
./inputDataset/EAN13-13mil/14cm/EAN13 13mil 14cm (38).bmp
./inputDataset/EAN13-13mil/14cm/EAN13 13mil 14cm (44).bmp
./inputDataset/EAN13-13mil/14cm/EAN13 13mil 14cm (1).bmp
./inputDataset/EAN13-13mil/14cm/EAN13 13mil 14cm (9).bmp
./inputDataset/EAN13-13mil/14cm/EAN13 13mil 14cm (43).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (12).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (13).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (1).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (18).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (20).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (17).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (2).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (23).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (21).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (19).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mil 5cm (15).bmp
./inputDataset/EAN13-13mil/5cm/EAN13 13mi

./inputDataset/mat25/picture_116.bmp
./inputDataset/mat25/picture_82.bmp
./inputDataset/mat25/picture_93.bmp
./inputDataset/mat25/picture_50.bmp
./inputDataset/mat25/picture_100.bmp
./inputDataset/mat25/picture_91.bmp
./inputDataset/mat25/picture_65.bmp
./inputDataset/mat25/picture_51.bmp
./inputDataset/mat25/picture_138.bmp
./inputDataset/mat25/picture_104.bmp
./inputDataset/mat25/picture_88.bmp
./inputDataset/mat25/picture_59.bmp
./inputDataset/mat25/000029.bmp
./inputDataset/mat25/picture_35.bmp
./inputDataset/mat25/picture_22.bmp
./inputDataset/mat25/picture_137.bmp
./inputDataset/mat25/picture_14.bmp
./inputDataset/mat25/picture_98.bmp
./inputDataset/mat25/picture_111.bmp
./inputDataset/mat25/picture_2.bmp
./inputDataset/mat25/picture_103.bmp
./inputDataset/mat25/picture_4.bmp
./inputDataset/mat25/picture_10.bmp
./inputDataset/mat25/picture_102.bmp
./inputDataset/mat25/picture_57.bmp
./inputDataset/mat25/picture_89.bmp
./inputDataset/mat25/picture_85.bmp
./inputDataset/mat25/pictu

## 随机剪裁

In [4]:
'''
定义裁剪函数，四个参数分别是：
左上角横坐标x0
左上角纵坐标y0
裁剪宽度w
裁剪高度h
'''
crop_image = lambda img, x0, y0, w, h: img[y0:y0+h, x0:x0+w]

In [5]:
'''
随机裁剪
area_ratio为裁剪画面占原画面的比例
hw_vari是扰动占原高宽比的比例范围
'''
def random_crop(img, area_ratio, hw_vari):
    h, w = img.shape[:2]
    hw_delta = np.random.uniform(-hw_vari, hw_vari)
    hw_mult = 1 + hw_delta
    
    # 下标进行裁剪，宽高必须是正整数
    w_crop = int(round(w*np.sqrt(area_ratio*hw_mult)))

    # 裁剪宽度不可超过原图可裁剪宽度
    if w_crop > w:
        w_crop = w

    h_crop = int(round(h*np.sqrt(area_ratio/hw_mult)))
    if h_crop > h:
        h_crop = h

    # 随机生成左上角的位置
    x0 = np.random.randint(0, w-w_crop+1)
    y0 = np.random.randint(0, h-h_crop+1)

    return crop_image(img, x0, y0, w_crop, h_crop)

In [6]:
file1 = "./tempDataset/step1-图像任意大小剪裁/"
mkdir(file1) 

# test
for root, dirs, files in os.walk(file0, topdown=False):
    i = 0
    for name in files:
        print((""+os.path.join(root, name)))
        img=cv2.imread((""+os.path.join(root, name)))
        area_ratio = 0.9
        hw_vari = 0.1
        i = i + 1
        cv2.imwrite(file1+ str(i) + '.bmp',
                    random_crop(img, area_ratio, hw_vari))
    for name in dirs:
        print(os.path.join(root, name))

---  new folder...  ---
---  OK  ---
./tempDataset/step0-重新编号后的数据集/360.bmp
./tempDataset/step0-重新编号后的数据集/770.bmp
./tempDataset/step0-重新编号后的数据集/352.bmp
./tempDataset/step0-重新编号后的数据集/1655.bmp
./tempDataset/step0-重新编号后的数据集/1328.bmp
./tempDataset/step0-重新编号后的数据集/206.bmp
./tempDataset/step0-重新编号后的数据集/782.bmp
./tempDataset/step0-重新编号后的数据集/1162.bmp
./tempDataset/step0-重新编号后的数据集/115.bmp
./tempDataset/step0-重新编号后的数据集/1001.bmp
./tempDataset/step0-重新编号后的数据集/1176.bmp
./tempDataset/step0-重新编号后的数据集/886.bmp
./tempDataset/step0-重新编号后的数据集/1465.bmp
./tempDataset/step0-重新编号后的数据集/621.bmp
./tempDataset/step0-重新编号后的数据集/1104.bmp
./tempDataset/step0-重新编号后的数据集/630.bmp
./tempDataset/step0-重新编号后的数据集/585.bmp
./tempDataset/step0-重新编号后的数据集/1455.bmp
./tempDataset/step0-重新编号后的数据集/247.bmp
./tempDataset/step0-重新编号后的数据集/1203.bmp
./tempDataset/step0-重新编号后的数据集/1016.bmp
./tempDataset/step0-重新编号后的数据集/1571.bmp
./tempDataset/step0-重新编号后的数据集/365.bmp
./tempDataset/step0-重新编号后的数据集/914.bmp
./tempDataset/step0-重新编号后的数据集/72.bmp
./t

./tempDataset/step0-重新编号后的数据集/219.bmp
./tempDataset/step0-重新编号后的数据集/1210.bmp
./tempDataset/step0-重新编号后的数据集/1429.bmp
./tempDataset/step0-重新编号后的数据集/591.bmp
./tempDataset/step0-重新编号后的数据集/958.bmp
./tempDataset/step0-重新编号后的数据集/1136.bmp
./tempDataset/step0-重新编号后的数据集/814.bmp
./tempDataset/step0-重新编号后的数据集/1518.bmp
./tempDataset/step0-重新编号后的数据集/979.bmp
./tempDataset/step0-重新编号后的数据集/32.bmp
./tempDataset/step0-重新编号后的数据集/1093.bmp
./tempDataset/step0-重新编号后的数据集/549.bmp
./tempDataset/step0-重新编号后的数据集/1046.bmp
./tempDataset/step0-重新编号后的数据集/1432.bmp
./tempDataset/step0-重新编号后的数据集/660.bmp
./tempDataset/step0-重新编号后的数据集/75.bmp
./tempDataset/step0-重新编号后的数据集/1525.bmp
./tempDataset/step0-重新编号后的数据集/310.bmp
./tempDataset/step0-重新编号后的数据集/1059.bmp
./tempDataset/step0-重新编号后的数据集/507.bmp
./tempDataset/step0-重新编号后的数据集/1074.bmp
./tempDataset/step0-重新编号后的数据集/815.bmp
./tempDataset/step0-重新编号后的数据集/1284.bmp
./tempDataset/step0-重新编号后的数据集/791.bmp
./tempDataset/step0-重新编号后的数据集/154.bmp
./tempDataset/step0-重新编号后的数据集/1532.bmp
./

./tempDataset/step0-重新编号后的数据集/50.bmp
./tempDataset/step0-重新编号后的数据集/1036.bmp
./tempDataset/step0-重新编号后的数据集/999.bmp
./tempDataset/step0-重新编号后的数据集/461.bmp
./tempDataset/step0-重新编号后的数据集/771.bmp
./tempDataset/step0-重新编号后的数据集/925.bmp
./tempDataset/step0-重新编号后的数据集/831.bmp
./tempDataset/step0-重新编号后的数据集/1041.bmp
./tempDataset/step0-重新编号后的数据集/191.bmp
./tempDataset/step0-重新编号后的数据集/131.bmp
./tempDataset/step0-重新编号后的数据集/1106.bmp
./tempDataset/step0-重新编号后的数据集/164.bmp
./tempDataset/step0-重新编号后的数据集/128.bmp
./tempDataset/step0-重新编号后的数据集/1568.bmp
./tempDataset/step0-重新编号后的数据集/51.bmp
./tempDataset/step0-重新编号后的数据集/1370.bmp
./tempDataset/step0-重新编号后的数据集/280.bmp
./tempDataset/step0-重新编号后的数据集/1220.bmp
./tempDataset/step0-重新编号后的数据集/1273.bmp
./tempDataset/step0-重新编号后的数据集/1417.bmp
./tempDataset/step0-重新编号后的数据集/1281.bmp
./tempDataset/step0-重新编号后的数据集/281.bmp
./tempDataset/step0-重新编号后的数据集/398.bmp
./tempDataset/step0-重新编号后的数据集/1178.bmp
./tempDataset/step0-重新编号后的数据集/860.bmp
./tempDataset/step0-重新编号后的数据集/781.bmp
./te

./tempDataset/step0-重新编号后的数据集/1266.bmp
./tempDataset/step0-重新编号后的数据集/1463.bmp
./tempDataset/step0-重新编号后的数据集/650.bmp
./tempDataset/step0-重新编号后的数据集/29.bmp
./tempDataset/step0-重新编号后的数据集/1194.bmp
./tempDataset/step0-重新编号后的数据集/931.bmp
./tempDataset/step0-重新编号后的数据集/1383.bmp
./tempDataset/step0-重新编号后的数据集/1385.bmp
./tempDataset/step0-重新编号后的数据集/649.bmp
./tempDataset/step0-重新编号后的数据集/1045.bmp
./tempDataset/step0-重新编号后的数据集/1250.bmp
./tempDataset/step0-重新编号后的数据集/597.bmp
./tempDataset/step0-重新编号后的数据集/381.bmp
./tempDataset/step0-重新编号后的数据集/1202.bmp
./tempDataset/step0-重新编号后的数据集/1098.bmp
./tempDataset/step0-重新编号后的数据集/229.bmp
./tempDataset/step0-重新编号后的数据集/224.bmp
./tempDataset/step0-重新编号后的数据集/143.bmp
./tempDataset/step0-重新编号后的数据集/1348.bmp
./tempDataset/step0-重新编号后的数据集/857.bmp
./tempDataset/step0-重新编号后的数据集/476.bmp
./tempDataset/step0-重新编号后的数据集/561.bmp
./tempDataset/step0-重新编号后的数据集/1349.bmp
./tempDataset/step0-重新编号后的数据集/1159.bmp
./tempDataset/step0-重新编号后的数据集/544.bmp
./tempDataset/step0-重新编号后的数据集/622.bmp
.

./tempDataset/step0-重新编号后的数据集/170.bmp
./tempDataset/step0-重新编号后的数据集/1426.bmp
./tempDataset/step0-重新编号后的数据集/1471.bmp
./tempDataset/step0-重新编号后的数据集/748.bmp
./tempDataset/step0-重新编号后的数据集/1013.bmp
./tempDataset/step0-重新编号后的数据集/1034.bmp
./tempDataset/step0-重新编号后的数据集/980.bmp
./tempDataset/step0-重新编号后的数据集/862.bmp
./tempDataset/step0-重新编号后的数据集/1114.bmp
./tempDataset/step0-重新编号后的数据集/1042.bmp
./tempDataset/step0-重新编号后的数据集/488.bmp
./tempDataset/step0-重新编号后的数据集/1022.bmp
./tempDataset/step0-重新编号后的数据集/533.bmp
./tempDataset/step0-重新编号后的数据集/1246.bmp
./tempDataset/step0-重新编号后的数据集/447.bmp
./tempDataset/step0-重新编号后的数据集/221.bmp
./tempDataset/step0-重新编号后的数据集/1442.bmp
./tempDataset/step0-重新编号后的数据集/920.bmp
./tempDataset/step0-重新编号后的数据集/1440.bmp
./tempDataset/step0-重新编号后的数据集/481.bmp
./tempDataset/step0-重新编号后的数据集/612.bmp
./tempDataset/step0-重新编号后的数据集/63.bmp
./tempDataset/step0-重新编号后的数据集/564.bmp
./tempDataset/step0-重新编号后的数据集/1195.bmp
./tempDataset/step0-重新编号后的数据集/837.bmp
./tempDataset/step0-重新编号后的数据集/1561.bmp
.

./tempDataset/step0-重新编号后的数据集/253.bmp
./tempDataset/step0-重新编号后的数据集/755.bmp
./tempDataset/step0-重新编号后的数据集/1352.bmp
./tempDataset/step0-重新编号后的数据集/921.bmp
./tempDataset/step0-重新编号后的数据集/326.bmp
./tempDataset/step0-重新编号后的数据集/385.bmp
./tempDataset/step0-重新编号后的数据集/1591.bmp
./tempDataset/step0-重新编号后的数据集/1058.bmp
./tempDataset/step0-重新编号后的数据集/803.bmp
./tempDataset/step0-重新编号后的数据集/292.bmp
./tempDataset/step0-重新编号后的数据集/531.bmp
./tempDataset/step0-重新编号后的数据集/1515.bmp
./tempDataset/step0-重新编号后的数据集/712.bmp
./tempDataset/step0-重新编号后的数据集/933.bmp
./tempDataset/step0-重新编号后的数据集/26.bmp
./tempDataset/step0-重新编号后的数据集/1607.bmp
./tempDataset/step0-重新编号后的数据集/691.bmp
./tempDataset/step0-重新编号后的数据集/1207.bmp
./tempDataset/step0-重新编号后的数据集/858.bmp
./tempDataset/step0-重新编号后的数据集/307.bmp
./tempDataset/step0-重新编号后的数据集/293.bmp
./tempDataset/step0-重新编号后的数据集/1170.bmp
./tempDataset/step0-重新编号后的数据集/1599.bmp
./tempDataset/step0-重新编号后的数据集/1534.bmp
./tempDataset/step0-重新编号后的数据集/469.bmp
./tempDataset/step0-重新编号后的数据集/872.bmp
./te

./tempDataset/step0-重新编号后的数据集/368.bmp
./tempDataset/step0-重新编号后的数据集/516.bmp
./tempDataset/step0-重新编号后的数据集/847.bmp
./tempDataset/step0-重新编号后的数据集/1512.bmp
./tempDataset/step0-重新编号后的数据集/456.bmp
./tempDataset/step0-重新编号后的数据集/798.bmp
./tempDataset/step0-重新编号后的数据集/1615.bmp
./tempDataset/step0-重新编号后的数据集/769.bmp
./tempDataset/step0-重新编号后的数据集/451.bmp
./tempDataset/step0-重新编号后的数据集/1307.bmp
./tempDataset/step0-重新编号后的数据集/1215.bmp
./tempDataset/step0-重新编号后的数据集/1131.bmp
./tempDataset/step0-重新编号后的数据集/1466.bmp
./tempDataset/step0-重新编号后的数据集/881.bmp
./tempDataset/step0-重新编号后的数据集/1102.bmp
./tempDataset/step0-重新编号后的数据集/431.bmp
./tempDataset/step0-重新编号后的数据集/194.bmp
./tempDataset/step0-重新编号后的数据集/1304.bmp
./tempDataset/step0-重新编号后的数据集/136.bmp
./tempDataset/step0-重新编号后的数据集/633.bmp
./tempDataset/step0-重新编号后的数据集/1188.bmp
./tempDataset/step0-重新编号后的数据集/967.bmp
./tempDataset/step0-重新编号后的数据集/899.bmp
./tempDataset/step0-重新编号后的数据集/1163.bmp
./tempDataset/step0-重新编号后的数据集/1436.bmp
./tempDataset/step0-重新编号后的数据集/866.bmp
.

./tempDataset/step0-重新编号后的数据集/668.bmp
./tempDataset/step0-重新编号后的数据集/1186.bmp
./tempDataset/step0-重新编号后的数据集/1611.bmp
./tempDataset/step0-重新编号后的数据集/349.bmp
./tempDataset/step0-重新编号后的数据集/1448.bmp
./tempDataset/step0-重新编号后的数据集/1585.bmp
./tempDataset/step0-重新编号后的数据集/258.bmp
./tempDataset/step0-重新编号后的数据集/313.bmp
./tempDataset/step0-重新编号后的数据集/1339.bmp
./tempDataset/step0-重新编号后的数据集/1450.bmp
./tempDataset/step0-重新编号后的数据集/1218.bmp
./tempDataset/step0-重新编号后的数据集/765.bmp
./tempDataset/step0-重新编号后的数据集/1161.bmp
./tempDataset/step0-重新编号后的数据集/1206.bmp
./tempDataset/step0-重新编号后的数据集/1369.bmp
./tempDataset/step0-重新编号后的数据集/377.bmp
./tempDataset/step0-重新编号后的数据集/312.bmp
./tempDataset/step0-重新编号后的数据集/1545.bmp
./tempDataset/step0-重新编号后的数据集/1511.bmp
./tempDataset/step0-重新编号后的数据集/44.bmp
./tempDataset/step0-重新编号后的数据集/1456.bmp
./tempDataset/step0-重新编号后的数据集/186.bmp
./tempDataset/step0-重新编号后的数据集/1578.bmp
./tempDataset/step0-重新编号后的数据集/818.bmp
./tempDataset/step0-重新编号后的数据集/653.bmp
./tempDataset/step0-重新编号后的数据集/1492.bm

## 旋转

In [7]:
file2 = "./tempDataset/step2-图像任意角度旋转/"
mkdir(file2) 

def rotate_image(img, angle, crop):
    h, w = img.shape[:2]
	
	# 旋转角度的周期是360°
    angle %= 360
	
	# 用OpenCV内置函数计算仿射矩阵
    M_rotate = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
	
	# 得到旋转后的图像
    img_rotated = cv2.warpAffine(img, M_rotate, (w, h))

	# 如果需要裁剪去除黑边
    if crop:
        angle_crop = angle % 180             	    # 对于裁剪角度的等效周期是180°
        if angle_crop > 90:                        	# 并且关于90°对称
            angle_crop = 180 - angle_crop
		
        theta = angle_crop * np.pi / 180.0    		# 转化角度为弧度
        hw_ratio = float(h) / float(w)    		    # 计算高宽比
		

        tan_theta = np.tan(theta)                   # 计算裁剪边长系数的分子项
        numerator = np.cos(theta) + np.sin(theta) * tan_theta
		

        r = hw_ratio if h > w else 1 / hw_ratio		# 计算分母项中和宽高比相关的项
        denominator = r * tan_theta + 1		 		# 计算分母项

        crop_mult = numerator / denominator			# 计算最终的边长系数
        w_crop = int(round(crop_mult*w))			# 得到裁剪区域
        h_crop = int(round(crop_mult*h))
        x0 = int((w-w_crop)/2)
        y0 = int((h-h_crop)/2)
        img_rotated = crop_image(img_rotated, x0, y0, w_crop, h_crop)
    return img_rotated

---  new folder...  ---
---  OK  ---


In [8]:
# test
i = 0
for root, dirs, files in os.walk(file1, topdown=False):
    for name in files:
        img=cv2.imread((""+os.path.join(root, name)))
        angle = 180
        crop = 0.1
        rotatedImg = rotate_image(img, angle, crop)
        i = i + 1
        cv2.imwrite(file2 + str(i) + '.bmp',rotatedImg)
    for name in dirs:
        print(os.path.join(root, name))

## 伽马变换

In [9]:
'''
定义gamma变换函数：
gamma就是Gamma
'''
def gamma_transform(img, gamma):
    gamma_table = [np.power(x / 255.0, gamma) * 255.0 for x in range(256)]
    gamma_table = np.round(np.array(gamma_table)).astype(np.uint8)
    return cv2.LUT(img, gamma_table)

'''
随机gamma变换
gamma_vari是Gamma变化的范围[1/gamma_vari, gamma_vari)
'''
def random_gamma_transform(img, gamma_vari):
    log_gamma_vari = np.log(gamma_vari)
    alpha = np.random.uniform(-log_gamma_vari, log_gamma_vari)
    gamma = np.exp(alpha)
    return gamma_transform(img, gamma)

In [10]:
file3 = "./tempDataset/step3-图像伽马变换/"
mkdir(file3) 

---  new folder...  ---
---  OK  ---


In [11]:
# test
img = cv2.imread(inputDataset)

# test
i = 0
for root, dirs, files in os.walk(file2, topdown=False):
    for name in files:
        img=cv2.imread((""+os.path.join(root, name)))
        gamma_vari = 0.8
        gammaedImg = random_gamma_transform(img, gamma_vari)
        i = i + 1
        cv2.imwrite(file3 + str(i) + '.bmp',gammaedImg)
    for name in dirs:
        print(os.path.join(root, name))

## 图像模糊

In [14]:
file4 = "./tempDataset/step4-高斯模糊/"
mkdir(file4) 

# test
i = 0
for root, dirs, files in os.walk(file3, topdown=False):
    for name in files:
        img=cv2.imread((""+os.path.join(root, name)))
        blurImg = cv2.GaussianBlur(img,(3,3),1)
        i = i + 1
        cv2.imwrite(file4 + str(i) + '.bmp',blurImg)
    for name in dirs:
        print(os.path.join(root, name))

---  There is this folder!  ---


## 图像resize得到输出

In [17]:
file5 = "./outDataset/"
mkdir(file5) 

i = 0
for root, dirs, files in os.walk("./tempDataset/", topdown=False):    
    for name in files:
        img=cv2.imread((""+os.path.join(root, name)))
        dst=cv2.resize(img,(640,480)) #改变图片尺寸
        dst = cv2.cvtColor(dst, cv2.COLOR_RGB2GRAY)
        i = i + 1
        cv2.imwrite(file5+ str(i) + '.bmp',dst)
    for name in dirs:
        print(os.path.join(root, name))

---  new folder...  ---
---  OK  ---
./tempDataset/step0-重新编号后的数据集
./tempDataset/step1-图像任意大小剪裁
./tempDataset/step4-高斯模糊
./tempDataset/step3-图像伽马变换
./tempDataset/step2-图像任意角度旋转
